In [1]:
from dotenv import load_dotenv

load_dotenv()


True

In [2]:
from llama_index.core import SimpleDirectoryReader
import os

# load documents
documents = SimpleDirectoryReader('./paul_graham/').load_data()
len(documents)


1

In [3]:
from llama_index.core.node_parser import SentenceSplitter


# Initialize the sentence splitter with desired parameters
node_parser = SentenceSplitter(chunk_size=512, chunk_overlap=64)

# Assuming 'documents' is a list of Document objects
nodes = node_parser.get_nodes_from_documents(documents)

In [3]:
from llama_index.vector_stores.deeplake import DeepLakeVectorStore

myid = "charanvardhan"
dataset_path = f"hub://{myid}/LlamaIndex_paulgraham_essay"

/Users/charanmannuru/miniconda3/envs/idx/lib/python3.11/site-packages/deeplake/util/check_latest_version.py:32: UserWarning: A newer version of deeplake (4.2.1) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(


In [5]:
from llama_index.core import StorageContext

storage_context = StorageContext.from_defaults(vector_store=vector_store)
storage_context.docstore.add_documents(nodes)


In [6]:
all_node_ids = list(storage_context.docstore.docs.keys())
print(f"Total nodes: {len(all_node_ids)}")
print("First 5 node IDs:", all_node_ids[:5])

Total nodes: 42
First 5 node IDs: ['48482c90-2850-4991-ad8a-a2fdf672be33', '8d518718-1bb3-477e-abb7-f3889b560609', '2166b23f-22dc-4c61-b279-5219b071c897', '0d823376-f78b-42c7-bb68-98effeb78a05', '885146f6-3496-4b75-9e88-d4b2c48d59e5']


## FaithfulnessEvaluator

In [4]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings


llm = OpenAI(
    temperature=0,
    model="gpt-3.5-turbo",
    max_tokens=512,
    streaming=True,
)
# Use Settings instead of ServiceContext
Settings.llm = llm


In [6]:
from llama_index.vector_stores.deeplake import DeepLakeVectorStore
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex

vector_store = DeepLakeVectorStore(dataset_path=dataset_path, overwrite=False, read_only=True)


Deep Lake Dataset in hub://charanvardhan/LlamaIndex_paulgraham_essay already exists, loading from the storage


In [9]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [10]:
index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)

In [11]:
from llama_index.core.evaluation import FaithfulnessEvaluator

evaluator = FaithfulnessEvaluator()

In [12]:
query_engine = index.as_query_engine()
print(type(query_engine))
response = query_engine.query("What does Paul Graham do?")

eval_result = evaluator.evaluate_response(response=response)

<class 'llama_index.core.query_engine.retriever_query_engine.RetrieverQueryEngine'>


In [13]:
print( "> response:", response )

print( "> evaluator result:", eval_result.passing )

> response: Paul Graham organizes a Summer Founders Program for undergraduates, where he funds selected groups of students to work on startup projects instead of taking traditional summer jobs. He also gives talks and writes essays to share his insights and advice on starting a startup. Additionally, he is involved in investing in startups and providing guidance to young founders.
> evaluator result: True
